<a href="https://colab.research.google.com/github/chaimaebouassab4-boop/Deep-Learning-Lab3-NLP-Sequence-Models/blob/main/lab3_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

#  main directories
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models/gpt2_finetuned', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Directory structure created!")

In [1]:
# تثبيت المكتبات اللازمة
!pip install transformers datasets accelerate arabert farasapy torch scikit-learn nltk

# استيراد المكتبات
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from arabert.preprocess import ArabertPreprocessor
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# النصوص العربية التقنية (موسعة وحديثة 2025)
texts = [
    "قالت وكالة الأنباء الألمانية إن شركة غوغل أغلقت ثغرتين أمنيتين تؤثران على نظام أندرويد... وتعتمد الثغرات على استغلال طبقات إطار العمل المختلفة لتطبيقات أندرويد.",
    "حفظ على خلاف نبوءات نهاية الوظائف، يكشف تقرير لصحيفة إيكونوميست أن الذكاء الاصطناعي لا يكتفي بعدم القضاء على العمل، بل يخلق فئات وظيفية جديدة... وأن المهارات البشرية أصبحت العملة الأعلى قيمة في سوق العمل الجديد.",
    "ووفقا لتقرير نشره موقع بي آر نيوز واير، من المتوقع أن يصل سوق تكنولوجيا البلوك تشين إلى 39.7 مليار دولار بحلول عام 2025...",
    "اختارت مجلة تايم الأميركية مهندسي الذكاء الاصطناعي شخصية عام 2025، وفي مقدمتهم جنسن هوانغ، الرئيس التنفيذي لشركة إنفيديا.",
    "أطلقت السعودية تطبيق HUMAIN Chat، أول تطبيق ذكاء اصطناعي حواري باللغة العربية، مدعوماً بنموذج ALLaM 34B.",
    "أعلنت أبوظبي وشركة Nvidia عن إطلاق مختبر مشترك للبحث في الذكاء الاصطناعي والروبوتات.",
    "توقعات 2025: سيطر الذكاء الاصطناعي والتهديدات العميقة Deepfakes على نقاشات الأمن السيبراني.",
    "في عام 2025، يعيد الذكاء الاصطناعي والحوسبة الكمومية تشكيل الأمن السيبراني.",
    "أعلنت بالو ألتو نتوركس أن 2025 عام الاضطراب السيبراني، مع زيادة الهجمات واسعة النطاق.",
    "كشفت تقارير عن هجمات فيشينغ تجاوزت 6.3 مليون محاولة في 2025 على منصات التسوق والألعاب."
]

scores = [9.5, 9.2, 8.0, 9.8, 9.7, 9.4, 9.0, 8.8, 9.1, 8.5]

df = pd.DataFrame({'Text': texts, 'Score': scores})

# حفظ في Colab (المسار الصحيح الآن)
df.to_csv('/content/arabic_tech_dataset.csv', index=False)

print(df)
print("\nتم حفظ الـ Dataset في /content/arabic_tech_dataset.csv")

                                                Text  Score
0  قالت وكالة الأنباء الألمانية إن شركة غوغل أغلق...    9.5
1  حفظ على خلاف نبوءات نهاية الوظائف، يكشف تقرير ...    9.2
2  ووفقا لتقرير نشره موقع بي آر نيوز واير، من الم...    8.0
3  اختارت مجلة تايم الأميركية مهندسي الذكاء الاصط...    9.8
4  أطلقت السعودية تطبيق HUMAIN Chat، أول تطبيق ذك...    9.7
5  أعلنت أبوظبي وشركة Nvidia عن إطلاق مختبر مشترك...    9.4
6  توقعات 2025: سيطر الذكاء الاصطناعي والتهديدات ...    9.0
7  في عام 2025، يعيد الذكاء الاصطناعي والحوسبة ال...    8.8
8  أعلنت بالو ألتو نتوركس أن 2025 عام الاضطراب ال...    9.1
9  كشفت تقارير عن هجمات فيشينغ تجاوزت 6.3 مليون م...    8.5

تم حفظ الـ Dataset في /content/arabic_tech_dataset.csv


In [3]:
model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)

def arabic_preprocess(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = arabert_prep.preprocess(text)
    return text

df['clean_text'] = df['Text'].apply(arabic_preprocess)
df.to_csv('/content/arabic_tech_preprocessed.csv', index=False)

tokenizer = AutoTokenizer.from_pretrained(model_name)

print("تم الـ Preprocessing بنجاح!")
print(df[['Text', 'clean_text', 'Score']].head())

[2025-12-18 23:17:01,746 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


تم الـ Preprocessing بنجاح!
                                                Text  \
0  قالت وكالة الأنباء الألمانية إن شركة غوغل أغلق...   
1  حفظ على خلاف نبوءات نهاية الوظائف، يكشف تقرير ...   
2  ووفقا لتقرير نشره موقع بي آر نيوز واير، من الم...   
3  اختارت مجلة تايم الأميركية مهندسي الذكاء الاصط...   
4  أطلقت السعودية تطبيق HUMAIN Chat، أول تطبيق ذك...   

                                          clean_text  Score  
0  قال +ت وكال +ة ال+ أنباء ال+ ألماني +ة إن شرك ...    9.5  
1  حفظ على خلاف نبوء +ات نهاي +ة ال+ وظائف يكشف ت...    9.2  
2  و+ وفق +ا ل+ تقرير نشر +ه موقع بي آر نيوز و+ ا...    8.0  
3  اختار +ت مجل +ة تايم ال+ أميركي +ة مهندس +ي ال...    9.8  
4  أطلق +ت ال+ سعودي +ة تطبيق HUMAIN Chat أول تطب...    9.7  


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
arabert_model = AutoModel.from_pretrained(model_name)

def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

texts_clean = df['clean_text'].tolist()
scores = df['Score'].values

encodings = tokenize_texts(texts_clean)

class TechDataset(Dataset):
    def __init__(self, encodings, scores):
        self.encodings = encodings
        self.scores = torch.tensor(scores, dtype=torch.float32)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.scores[idx]
        return item

    def __len__(self):
        return len(self.scores)

train_texts, val_texts, train_scores, val_scores = train_test_split(texts_clean, scores, test_size=0.25, random_state=42)

train_enc = tokenize_texts(train_texts)
val_enc = tokenize_texts(val_texts)

train_dataset = TechDataset(train_enc, train_scores)
val_dataset = TechDataset(val_enc, val_scores)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [7]:
class SequenceRegressor(nn.Module):
    def __init__(self, seq_type='lstm', hidden_size=256, num_layers=2, bidirectional=False, dropout=0.3):
        super().__init__()
        self.embedding = arabert_model.embeddings
        self.seq_type = seq_type
        self.bidirectional = bidirectional

        if seq_type == 'rnn':
            self.rnn = nn.RNN(768, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        elif seq_type == 'gru':
            self.rnn = nn.GRU(768, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        elif seq_type == 'lstm':
            self.rnn = nn.LSTM(768, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)

        mult = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_size * mult, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            emb = self.embedding(input_ids)

        mask = attention_mask.unsqueeze(-1).expand(emb.size()).float()
        emb = emb * mask

        out, _ = self.rnn(emb)
        out = out[:, -1, :]  # آخر حالة مخفية
        out = self.dropout(out)
        return self.fc(out).squeeze()

def train_and_evaluate(model_name, seq_type, bidirectional):
    model = SequenceRegressor(seq_type=seq_type, hidden_size=256, num_layers=2, bidirectional=bidirectional)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()

    for epoch in range(30):
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            preds = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            loss = loss_fn(preds, batch['labels'].to(device))
            loss.backward()
            optimizer.step()

    # تقييم
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for batch in val_loader:
            p = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
            preds.extend(p.cpu().numpy())
            true.extend(batch['labels'].numpy())

    mae = mean_absolute_error(true, preds)
    rmse = np.sqrt(mean_squared_error(true, preds))
    r2 = r2_score(true, preds)

    # BLEU بعد discretization
    def score_to_cat(s):
        if s >= 9: return "عالية جداً"
        elif s >= 8: return "عالية"
        elif s >= 7: return "جيدة"
        else: return "منخفضة"

    pred_cat = [score_to_cat(p) for p in preds]
    true_cat = [score_to_cat(t) for t in true]

    smoothie = SmoothingFunction().method4
    bleu = np.mean([sentence_bleu([ref.split()], hyp.split(), smoothing_function=smoothie) for ref, hyp in zip(true_cat, pred_cat)])

    print(f"{model_name} | MAE: {mae:.3f} | RMSE: {rmse:.3f} | R²: {r2:.3f} | BLEU: {bleu:.3f}")

    return mae

# تدريب وتقييم الأربعة
print("=== نتائج التقييم ===")
train_and_evaluate("RNN", 'rnn', False)
train_and_evaluate("Bi-RNN", 'rnn', True)
train_and_evaluate("Bi-GRU", 'gru', True)
train_and_evaluate("Bi-LSTM", 'lstm', True)

=== نتائج التقييم ===
RNN | MAE: 0.110 | RMSE: 0.149 | R²: -0.419 | BLEU: 0.221
Bi-RNN | MAE: 0.224 | RMSE: 0.259 | R²: -3.302 | BLEU: 0.319
Bi-GRU | MAE: 0.410 | RMSE: 0.470 | R²: -13.197 | BLEU: 0.319
Bi-LSTM | MAE: 0.254 | RMSE: 0.287 | R²: -4.298 | BLEU: 0.319


0.2536328633626302

In [6]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# كتابة النصوص في ملف للتدريب
full_text = "\n\n".join(df['Text'].tolist())

with open('/content/tech_corpus.txt', 'w', encoding='utf-8') as f:
    f.write(full_text)

# تحميل AraGPT2
gpt_model_name = "aubmindlab/aragpt2-base"
tokenizer_gpt = GPT2Tokenizer.from_pretrained(gpt_model_name)
tokenizer_gpt.pad_token = tokenizer_gpt.eos_token

model_gpt = GPT2LMHeadModel.from_pretrained(gpt_model_name)

# إعداد Dataset
train_dataset = TextDataset(tokenizer=tokenizer_gpt, file_path="/content/tech_corpus.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer_gpt, mlm=False)

# إعداد التدريب
training_args = TrainingArguments(
    output_dir="/content/aragpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=10,
    save_steps=100,
)

trainer = Trainer(
    model=model_gpt,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("بدء Fine-tuning...")
trainer.train()

# حفظ النموذج
model_gpt.save_pretrained("/content/aragpt2-finetuned")
tokenizer_gpt.save_pretrained("/content/aragpt2-finetuned")

/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


بدء Fine-tuning...


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

In [ ]:
model_gen = GPT2LMHeadModel.from_pretrained("/content/aragpt2-finetuned")
tokenizer_gen = GPT2Tokenizer.from_pretrained("/content/aragpt2-finetuned")
tokenizer_gen.pad_token = tokenizer_gen.eos_token

def generate_text(prompt, max_length=300):
    prompt_clean = arabert_prep.preprocess(prompt)
    inputs = tokenizer_gen(prompt_clean, return_tensors="pt")

    outputs = model_gen.generate(
        **inputs,
        max_length=max_length,
        num_beams=5,
        temperature=0.8,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    text = tokenizer_gen.decode(outputs[0], skip_special_tokens=True)
    text = arabert_prep.unpreprocess(text)
    return text

# اختبار
prompt = "في عام 2026، من المتوقع أن يصبح الذكاء الاصطناعي"
print(generate_text(prompt))